In [177]:
import pandas as pd

train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")
test_x = test_data.copy()

train_y = train_data["SalePrice"]
train_data.drop(columns=["SalePrice", "Id"], inplace=True)
test_data.drop(columns=["Id"], inplace=True)

In [178]:
## Splitting categorical and numerical values
ob_cols_tr = train_data.select_dtypes(include='object').columns
num_cols_tr = train_data.select_dtypes(include='number').columns

ob_cols_te = test_data.select_dtypes(include='object').columns
num_cols_te = test_data.select_dtypes(include='number').columns

## trian data after splitting
ob_tr = pd.DataFrame(train_data[ob_cols_tr])
num_tr = pd.DataFrame(train_data[num_cols_tr])

## test data after splitting
ob_te = pd.DataFrame(test_data[ob_cols_te])
num_te = pd.DataFrame(test_data[num_cols_te])

In [179]:
## imputing missing values of number values
from sklearn.impute import SimpleImputer

imp = SimpleImputer()
imp_tr = pd.DataFrame(imp.fit_transform(num_tr))
imp_te = pd.DataFrame(imp.transform(num_te))

imp_tr.columns = num_tr.columns
imp_te.columns = num_te.columns

In [180]:
## encoding categrical values
from sklearn.preprocessing import OrdinalEncoder

# replacing null values with 'Nothing'
ob_tr.fillna('Nothing', inplace=True)
ob_te.fillna('Nothing', inplace=True)

# combining train and test, because 'Nothing' won't be found in train data
con_tre = pd.concat([ob_tr, ob_te], axis=0)

enc = OrdinalEncoder()
enc_tre = pd.DataFrame(enc.fit_transform(con_tre)) # encoding together

# splitting
# enc_tr = enc_tre[:len(ob_tr)]
# enc_te = enc_tre[len(ob_tr):]

enc_tr = pd.DataFrame(enc_tre.iloc[:len(ob_tr)])
enc_te = pd.DataFrame(enc_tre.iloc[len(ob_tr):])
enc_te.reset_index(drop=True, inplace=True)

enc_tr.columns = ob_tr.columns
enc_te.columns = ob_te.columns

In [181]:
## combining numerical and categorical data
data_tr = pd.concat([imp_tr, enc_tr], axis=1)
data_te = pd.concat([imp_te, enc_te], axis=1)

In [182]:
from xgboost import XGBRegressor

# regression
model = XGBRegressor(n_estimators=1000, learning_rate=0.05)
model.fit(data_tr, train_y, verbose=False)
predictions = model.predict(data_te)

# printing output to the file
output = pd.DataFrame({"Id": test_x["Id"], "SalePrice": predictions})
output.to_csv("submission.csv", index=False)